In [1]:
# Library import

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from google.cloud import bigquery
from google.oauth2 import service_account

delta=dt.timedelta(days=1)

In [3]:
# Get main path and other folders
s_mainpath=Path.cwd()
print(s_mainpath)
s_parentpath=s_mainpath.parent.parent.parent
print(s_parentpath)
# folderpath=s_parentpath.joinpath(r'DatosXM\Despacho\\')

# Ruta General
sRutaDesp=r'C:\Información XM\\Publico\\'

sRutaPrint=s_mainpath.joinpath('Print')

c:\Alejo\Eje de Planeación\Análisis Eléctrico\ScriptsPython
c:\Alejo


In [4]:
# Rango de análisis

s_FechaIni='2025-01-27'
s_FechaFin='2025-02-09'

#Camniar a formato fecha
d_FechaIni=dt.datetime.strptime(s_FechaIni,'%Y-%m-%d')
d_FechaFin=dt.datetime.strptime(s_FechaFin,'%Y-%m-%d')

In [ ]:
# Inicializar fecha
d_Fecha=d_FechaIni

# Encabezados 
l_colDem=['Subarea'] 
l_colInt=['Subarea'] 

for i in range(1,25): 
    l_colDem.append(i)
    l_colInt.append(i)

# filetypes=['dSEGDES','dDEM','dINT','dLIM','dDEC'] # Lista con los archivos a analizar
# filetypes=['dDEC','dConFig','rDEC','rConFig'] # Lista con los archivos a analizar
filetypes=['dDEM','dINT'] # Lista con los archivos a analizar

# l_SubCar2=['ATLANTIC','BOLIVAR','GCM'] # Lista con las subareas

# l_SubCar=['ATLANTIC','BOLIVAR','GCM','CORDOSUC','CERROMAT'] # Lista con las subareas

df_Dem=pd.DataFrame()
df_Int=pd.DataFrame()
df_ConfR=pd.DataFrame()
df_GenR=pd.DataFrame()


while d_Fecha<=d_FechaFin:

    ano=d_Fecha.year
    mes=d_Fecha.month
    dia=d_Fecha.day

    for filetype in filetypes:

        if filetype=='dDEM':
            folderpath=Path(sRutaDesp + 'Despacho\\')
            file=filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
            s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
            df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colDem,encoding="ISO-8859-1")
            df_Aux['Fecha']=d_Fecha
            df_Aux2=df_Aux[df_Aux['Subarea'].isin(['BOLIVAR','ATLANTIC'])]
            # df_Aux2=df_Aux[df_Aux['Nombre'].isin(['TEBSA','FLORES IV','FLORES1'])]
            df_Dem=pd.concat([df_Dem,df_Aux2])

        elif filetype=='dINT':
            folderpath=Path(sRutaDesp + 'Despacho\\')
            file=filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
            s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
            df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colInt,encoding="ISO-8859-1")
            df_Aux['Fecha']=d_Fecha
            f_Aux2=df_Aux[df_Aux['Subarea'].isin(['BOLIVAR','ATLANTIC'])]
            df_Int=pd.concat([df_Int,f_Aux2])


    d_Fecha=d_Fecha+delta

In [6]:
# Función para asignar los días de la semana a cada fecha, si es festivo se trata como un domingo
import holidays
co_holidays = holidays.Colombia()

def typedays(row,weekday):

     if weekday==True:
          return row['Fecha'].weekday()
     else:
          if row['Fecha'] in co_holidays:
               return 'F'
          elif row['Fecha'].weekday()==5:
               return 'S'
          elif row['Fecha'].weekday()==6:
               return 'F'
          else:
               return 'O'

In [14]:
df_DemP=df_Dem.copy()
df_DemP=pd.melt(df_DemP, id_vars=['Fecha','Subarea'], var_name='Periodo', value_name='MW')

# Definir tipo de día
df_DemP['day_osf']=df_DemP.apply(lambda row: typedays(row,weekday=False),axis=1)

df_DemP=df_DemP.groupby(['Subarea','Periodo','day_osf'])[['MW']].mean().reset_index()

df_IntP=df_Int.copy()
df_IntP=pd.melt(df_IntP, id_vars=['Fecha','Subarea'], var_name='Periodo', value_name='MWInt')

# Definir tipo de día
df_IntP['day_osf']=df_IntP.apply(lambda row: typedays(row,weekday=False),axis=1)

df_IntP=df_IntP.groupby(['Subarea','Periodo','day_osf'])[['MWInt']].mean().reset_index()

data=df_DemP.merge(df_IntP,left_on=['Subarea','Periodo','day_osf'],right_on=['Subarea','Periodo','day_osf'], how='left')[['Subarea','Periodo','day_osf','MW','MWInt']]

data.to_csv(sRutaPrint.joinpath('DemInt.csv'),index=False)